In [1]:
#filename = "ml.py"
#exec(compile(open(filename, "rb").read(), filename, 'exec'))

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import dateutil.parser as parser
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

# read the dataset into pandas dataframe
# df = pd.read_csv('./../downsampled_data', delim_whitespace=False).dropna()
df = pd.read_csv('./../downsampled_data.csv').dropna()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/cameronyuen/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [23]:
df.columns

Index(['Unnamed: 0', 'video_id', 'title', 'publishedAt', 'channelId',
       'channelTitle', 'categoryId', 'duration', 'definition', 'trended_later',
       'time_retrieved', 'tags', 'thumbnail_link', 'ratings_disabled',
       'description', 'Channel_viewCount', 'Channel_subscriberCount',
       'Channel_hiddenSubscriberCount', 'Channel_videoCount', 'Channel_title',
       'Channel_description', 'Channel_publishedAt', 'Channel_country',
       'view_count_update_11_19_14', 'likes_update_11_19_14',
       'dislikes_update_11_19_14', 'comment_count_update_11_19_14',
       '11_19_14_update_timestamp', 'trending?', 'engagement_rate', 'INTL',
       'UNK', 'USA'],
      dtype='object')

In [24]:
df['categoryId'] = df['categoryId'].astype(np.float)

categories = pd.DataFrame(np.zeros((len(df), len(df['categoryId'].unique()))), columns = df['categoryId'].unique())
index = 0
for id in df['categoryId']:
    categories[id][index] = 1
    index += 1

In [25]:
# display the first five rows
retrieval_time = df['time_retrieved']
publish_time = df['publishedAt']
channel_publish_time = df['Channel_publishedAt']
retrieval_time_11_19_14 = df['11_19_14_update_timestamp']
columns_to_drop = ['Unnamed: 0', 'definition', 'categoryId', 'publishedAt', 'time_retrieved', 'Channel_title', '11_19_14_update_timestamp', 'Channel_publishedAt', 'video_id', 'channelId', 'thumbnail_link', 'Channel_country']
df = df.drop(columns_to_drop, axis = 1)

In [26]:
categories.columns

Float64Index([24.0, 25.0, 26.0, 17.0, 22.0, 20.0,  1.0, 10.0, 23.0, 28.0,  2.0,
              27.0, 19.0, 15.0, 30.0, 29.0],
             dtype='float64')

In [27]:
index = 0
for column in categories.columns:
    df.insert(df.shape[1], "Category_"+str(column), categories[column])
    index += 1

In [28]:
## time updates
import dateutil.parser as parser
age = []
age_update = []
channel_age = []
for i in df.index:
    channel_publish_time[i] = channel_publish_time[i].replace("\"", "")
    age.append(parser.isoparse(retrieval_time[i]) - parser.isoparse(publish_time[i]))
    age_update.append(parser.isoparse(retrieval_time_11_19_14[i]) - parser.isoparse(publish_time[i]))
    channel_age.append(parser.isoparse(channel_publish_time[i]) - parser.isoparse(publish_time[i]))
    if df['ratings_disabled'][i] == 'True':
        df['ratings_disabled'][i] = True
    elif df['ratings_disabled'][i] == 'False':
        df['ratings_disabled'][i] = False
        
    if df['Channel_hiddenSubscriberCount'][i] == 'True':
        df['Channel_hiddenSubscriberCount'][i] = True
    elif df['Channel_hiddenSubscriberCount'][i] == 'False':
        df['Channel_hiddenSubscriberCount'][i] = False
        
        
    if df['trended_later'][i] == 'True':
        df['trended_later'][i] = True
    elif df['trended_later'][i] == 'False':
        df['trended_later'][i] = False

<ipython-input-28-25b8cbbe352f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ratings_disabled'][i] = False
<ipython-input-28-25b8cbbe352f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Channel_hiddenSubscriberCount'][i] = False
<ipython-input-28-25b8cbbe352f>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trended_later'][i] = False
<ipython-input-28-25b8cbbe352f>:23: SettingWithCopyWarning: 
A value is trying to

In [29]:
## sentiment values

titles = df['title']
channel_title = df['channelTitle']
description = df['description']
channel_description = df['Channel_description']

title_sentiment_vals = []
channel_title_sentiment_vals = []
description_sentiment_vals = []
channel_description_sentiment_vals = []

sid = SentimentIntensityAnalyzer()
for sentence in titles:
    ss = sid.polarity_scores(str(sentence))
    title_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in channel_title:
    ss = sid.polarity_scores(str(sentence))
    channel_title_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in description:
    ss = sid.polarity_scores(str(sentence))
    description_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in channel_description:
    ss = sid.polarity_scores(str(sentence))
    channel_description_sentiment_vals.append(ss['pos']-ss['neg'])

In [30]:
df

,title,channelTitle,duration,trended_later,tags,ratings_disabled,description,Channel_viewCount,Channel_subscriberCount,Channel_hiddenSubscriberCount,Channel_videoCount,Channel_description,view_count_update_11_19_14,likes_update_11_19_14,dislikes_update_11_19_14,comment_count_update_11_19_14,trending?,engagement_rate,INTL,UNK,USA,Category_24.0,Category_25.0,Category_26.0,Category_17.0,Category_22.0,Category_20.0,Category_1.0,Category_10.0,Category_23.0,Category_28.0,Category_2.0,Category_27.0,Category_19.0,Category_15.0,Category_30.0,Category_29.0
0,Biden Victory Cold Open - SNL,Saturday Night Live,-0.0707230866960673,False,36.0,False,Joe Biden (Jim Carrey) and Donald Trump (Alec ...,0.819940,0.504608,False,-0.190170,"""Welcome to the official Saturday Night Live c...",6.152175,3.391273,10.316363,2.855296,True,0.028492,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,How President Trump and the White House reacte...,CNBC Television,-0.0856798827419832,False,28.0,False,CNBC's Kayla Tausche joins The News with Shepa...,-0.274970,-0.350255,False,1.155772,"""""",3.001911,0.578677,4.999903,3.142250,True,0.090361,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Watch Live: President-Elect Joe Biden Addresse...,ABC News,0.09488158288091483,False,13.0,False,#Election2020 #Election #JoeBiden #PresidentEl...,0.650191,0.461214,False,1.266836,"""Make ABC News your daily news outlet for brea...",1.368339,0.537524,2.314506,1.066382,True,0.006721,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,I GOT PLASTIC SURGERY TO SEE HOW MY FRIENDS RE...,Alexa Rivera,-0.07104706061763949,False,0.0,False,"This was definitely such a fun prank, their re...",-0.258217,0.002973,False,-0.408313,"""""",1.882086,2.655548,2.774959,0.721674,True,0.045112,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FC Barcelona 5 - 2 Real Betis - HIGHLIGHTS & G...,beIN SPORTS USA,-0.07288291283988188,False,12.0,False,Lionel Messi entró al segundo tiempo y resucit...,-0.330711,-0.407708,False,-0.204876,"""The fastest growing network in the US offerin...",0.280922,-0.071268,0.049159,0.022729,True,0.029918,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,"Eying WH, Bullish Biden Campaign Tells Trump T...",MSNBC,-0.09237534378780837,False,32.0,False,As experts worry Trump will refuse to concede ...,0.202458,-0.117663,False,0.633282,"""The official MSNBC YouTube Channel. MSNBC is...",-0.157203,-0.163979,-0.176888,-0.062852,False,0.000939,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876,Belize: Joko gegen die Kokosnuss | Duell um di...,Joko & Klaas,-0.06953518231696929,False,26.0,False,Joko zerdeppert sich seinen Kopf an einer Koko...,-0.235549,-0.325954,False,-0.380757,"""Joko und Klaas – Hier findet ihr alle neuen u...",-0.122305,-0.123057,-0.019108,-0.106405,False,0.004681,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
877,Alex De Minaur vs Ugo Humbert | Antwerp 2020 F...,Tennis TV,-0.07223496499673751,False,25.0,False,Two young guns battle it out for the Antwerp t...,-0.300937,-0.396426,False,-0.298508,"""The official Tennis TV YouTube channel, home ...",-0.151382,-0.164597,-0.192561,-0.105357,False,0.005805,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
878,Pikmin 3 Deluxe: Top 5 Things to Know Before Y...,Nintendo,0.07792694765197046,False,43.0,False,With only a week to go before the Pikmin™ 3 De...,-0.063224,0.192171,False,-0.277092,"""Welcome to the official Nintendo YouTube chan...",-0.205556,-0.179888,-0.172012,-0.093253,False,0.000266,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
from sklearn import preprocessing

df['title'] = title_sentiment_vals
df['channelTitle'] = channel_title_sentiment_vals
df['description'] = description_sentiment_vals
df['Channel_description'] = channel_description_sentiment_vals

le = preprocessing.LabelEncoder()
df['trending?'] = le.fit_transform(df['trending?'])
df['trended_later'] = le.fit_transform(df['trended_later'])
df['ratings_disabled'] = le.fit_transform(df['ratings_disabled'])
df['Channel_hiddenSubscriberCount'] = le.fit_transform(df['Channel_hiddenSubscriberCount'])
df[ 'duration'] = [float(i) for i in df[ 'duration']] 
pd.set_option('display.max_columns', None)
df = df.dropna()

In [32]:
# dummy values for now - get these interactively later...
import tensorflow as tf

best_model = False
if best_model:
    num_hidden_layers = 3
    num_hidden_layer_nodes = [20, 30, 20]
    train_ratio = .7
    hidden_layer_activations = ['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid']
    optimizer = 'adam'
    learning_rate = .005
    loss = 'mean_squared_error'
    metrics = [tf.keras.metrics.Accuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
    metrics_names = ["accuracy","recall","precision"]
    epochs = 300
    batch_size = 200
else:
    # build a custom model
    num_hidden_layers = 3
    num_hidden_layer_nodes = [20, 30, 20]
    train_ratio = .7
    hidden_layer_activations = ['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid']
    optimizer = 'adam'
    learning_rate = .005
    loss = 'mean_squared_error'
    metrics = [tf.keras.metrics.Accuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
    metrics_names = ["accuracy","recall","precision"]
    epochs = 300
    batch_size = 200

In [33]:
# Split the data into training and testing set by 70:30
ratio = 0.7
train, test = train_test_split(df, train_size=ratio, random_state=42)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

In [34]:
# separate data into x and y - just random y for now..
train_X = train.loc[:,train.columns != 'trending?']
train_Y = train['trending?']
test_X = test.loc[:,test.columns != 'trending?']
test_Y = test['trending?']

In [35]:
# build the logistic regression model - need clean data...
from sklearn.linear_model import LogisticRegression
LR_model = LogisticRegression(multi_class='ovr')
LR_model.fit(train_X, train_Y)

/Users/cameronyuen/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='ovr')

In [36]:
# confusion matrix
from sklearn.metrics import confusion_matrix

pred_Y = LR_model.predict(test_X);
confusion_matrix(test_Y, pred_Y)

array([[134,   9],
       [ 22,  44]])

In [37]:
# evaluate the accuracy of the LR model
accuracy = LR_model.score(test_X, test_Y)
print(accuracy)

0.8516746411483254


In [38]:
# build the ANN model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

ANN_model = keras.Sequential()

# add hidden layers
for i in range(num_hidden_layers):
    if i == 0:
        ANN_model.add(Dense(num_hidden_layer_nodes[i], input_dim = train_X.shape[1], activation=hidden_layer_activations[i]))
    ANN_model.add(Dense(num_hidden_layer_nodes[i], activation=hidden_layer_activations[i]))

# add output layers
ANN_model.add(Dense(1, activation=hidden_layer_activations[num_hidden_layers]))

ANN_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
ANN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 20)                740       
_________________________________________________________________
dense_6 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_7 (Dense)              (None, 30)                630       
_________________________________________________________________
dense_8 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 21        
Total params: 2,431
Trainable params: 2,431
Non-trainable params: 0
_________________________________________________________________


In [39]:
# train the model
ANN_model.fit(train_X, train_Y, validation_data=(test_X, test_Y), epochs=epochs, batch_size=batch_size)

Epoch 1/300
3/3 [==============================] - 0s 94ms/step - loss: 0.2447 - accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.2399 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 2/300
3/3 [==============================] - 0s 8ms/step - loss: 0.2335 - accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.2327 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 3/300
3/3 [==============================] - 0s 8ms/step - loss: 0.2238 - accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.2270 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 4/300
3/3 [==============================] - 0s 7ms/step - loss: 0.2152 - accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.2226 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_precision_1

In [40]:
# confusion matrix
pred_Y = ANN_model.predict_classes(test_X);
confusion_matrix(test_Y, pred_Y)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[128,  15],
       [ 15,  51]])

In [41]:
# report evaluation metrics 
evaluated_metrics = ANN_model.evaluate(test_X, test_Y)
for i in range(len(metrics)):
    print(metrics_names[i] + ": %.2f" % evaluated_metrics[i])

7/7 [==============================] - 0s 1ms/step - loss: 0.1047 - accuracy: 0.0000e+00 - recall_1: 0.7727 - precision_1: 0.7727
accuracy: 0.10
recall: 0.00
precision: 0.77
